In [1]:
import h5py
import numpy as np

## Initial values
Select the default values for the solar wind and ionospheric parameters

In [2]:
TempVal = 1.0e+5     #Default temperature, K                                                                                                                                                       
DenVal = 5          #Default density, #/cc                                                                                                                                                               
VxVal = 400.0     #Default wind, km/s                                                                                                                                                                  
F107Val = 100.0 #Default f10.7 flux                                                                                                                                                                  
TiltVal = 0.0      #Default dipole tilt, radians                                                                                                                                                        
mjdVal = 51623.0  #Default MJD, set for 2000-03-20 00:00:00 

Set start and end time in hours and delta between steps in solar wind file in seconds

In [3]:
startTime = 0 # Hours
endTime = 8 # Hours
dT = 60.0 #Seconds

In [4]:
simTime = (endTime-startTime)*3600.0
numSteps = np.int(np.ceil(simTime/dT)+1)

In [5]:
time = np.linspace(startTime,endTime,numSteps)

## Set output arrays to initial values

All variables need to be see including all componets of vectors.  For this simple setup we are going to only have Vx and no magnetic field.  Note, the solar wind file is in GSM coordinates so an Earthward directed solar wind is negative Vx.

In [6]:
Den = np.zeros(numSteps)+DenVal
Temp = np.zeros(numSteps)+TempVal
Vx = np.zeros(numSteps)-VxVal
Vy = np.zeros(numSteps)
Vz = np.zeros(numSteps)
Bx = np.zeros(numSteps)
By = np.zeros(numSteps)
Bz = np.zeros(numSteps)
F107 = np.zeros(numSteps)+F107Val
Tilt = np.zeros(numSteps)+TiltVal
Symh = np.zeros(numSteps)


Modified Julian Date needs to increase from the starting date.  MJD is in fraction of day so to create the MJD array we add time, which is currently in hours so it needs to be devided by 24, to the MJD starting value

In [7]:
mjd = mjdVal+time/24.0

## Alter solar wind parameters

Now we want to solar wind conditions that different from the default initial values.  In this case we are going to start with 2 hours of southward IMF $B_Z$ and then 2 hours northward followed by southward for the duration of the run. 

In [8]:
Bz[np.where(time<2.0)]=-5.0
Bz[np.where(time>2.0)] = 5.0
Bz[np.where(time>4.0)] = -5.0

## Magnetic Field Coefficents
Due do the boundary condition constraints $B_X$ cannot be any arbitary value.  We use $B_X = Bx0 + ByC*B_Y + BzC*B_Z$ as the formula for $B_X$ so we need to set those values before writing the solar wind file. 

In [9]:
Bx0 = 0.0
ByC = 0.0
BzC = 0.0

## Write Solar wind File

Kaiju uses HDF5 for it's solar wind input file.  The file is traditionally called bcwind.h5. Before writing to the file we will need to convert from traditonal units to code input units. 

In [10]:
TimeScl = 3600.0 #hrs -> sec
VScl = 1.0e3 #km/s -> m/s
BScl = 1.0 #nT
DenScl = 1.0 # cc
TempScl = 1.0 #K

In [11]:
fname = 'bcwind.h5'

In [12]:
with h5py.File(fname,'w') as hf:
    hf.create_dataset("T",data=TimeScl*time)
    hf.create_dataset("D",data=Den*DenScl)
    hf.create_dataset("Temp",data=Temp*TempScl)
    hf.create_dataset("Vx",data=Vx*VScl)
    hf.create_dataset("Vy",data=Vy*VScl)
    hf.create_dataset("Vz",data=Vz*VScl)
    hf.create_dataset("Bx",data=Bx*BScl)
    hf.create_dataset("By",data=By*BScl)
    hf.create_dataset("Bz",data=Bz*BScl)
    hf.create_dataset("tilt",data=Tilt)
    hf.create_dataset("f10.7",data=F107)
    hf.create_dataset("MJD",data=mjd)
    hf.create_dataset("symh",data=Symh)
    hf.create_dataset("Bx0",data=Bx0)
    hf.create_dataset("ByC",data=ByC)
    hf.create_dataset("BzC",data=BzC)